<a target="_blank" href="https://colab.research.google.com/github/bettercodepaul/data2day_2023_polars/blob/main/data2day_2023_Polars_Teil_1.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Polars: Der Turbo Boost für Dataframes

In diesem Notebook lernen wir Polars kennen. Polars ist eine extrem schnelle Dataframe-Bibliothek bzw. In-Memory-Abfrage-Engine. Sie zeichnet sich aus durch extrem parallele Ausführung, cache-effiziente Algorithmen und eine ausdrucksstarke API. Dadurch ist sie perfekt für die effiziente Abfrage und Transformation von Daten.

Polars ist in Rust geschrieben, nutzt das spalten-orientierte Format von Apache Arrow und besitzt eine Python-API.

Mehr Information gibt es hier:

- Homepage von Polars: https://www.pola.rs/
- User-Guide: https://pola-rs.github.io/polars/user-guide/
- API-Referenz: https://pola-rs.github.io/polars/py-polars/html/reference


## Installation + Vorbereitung

In [1]:
import urllib.request
import os.path

In [ ]:
# requirements.txt mit benötigten Bibliotheken laden
REQUIREMENTS_URL = "https://github.com/bettercodepaul/data2day_2023_polars/raw/main/requirements.txt"
urllib.request.urlretrieve(REQUIREMENTS_URL, os.path.basename(REQUIREMENTS_URL))

In [ ]:
# nicht vergessen, dass die Laufzeitumgebung ggf. neu gestartet werden muss
!pip install -qr requirements.txt

In [2]:
# Polars importieren
import polars as pl

In [ ]:
# bis zu 60 Zeichen pro Spalte ausgeben und Fließkommazahlen nicht abkürzen
pl.Config(fmt_str_lengths=60, fmt_float="full")

In [ ]:
# CSV Daten herunterladen
DATA_URL = "https://github.com/bettercodepaul/data2day_2023_polars/raw/main/spotify-charts-2017-2021-global-top200.csv.gz"
LOCAL_DATA_FILE_NAME = os.path.basename(DATA_URL)
urllib.request.urlretrieve(DATA_URL, LOCAL_DATA_FILE_NAME)

In [ ]:
# Übungen und Hilfsfunktionen herunterladen
EXERCISES_URL = "https://github.com/bettercodepaul/data2day_2023_polars/raw/main/data2day_exercises.py"
urllib.request.urlretrieve(EXERCISES_URL, os.path.basename(EXERCISES_URL))

In [ ]:
# Übungen und Hilfsfunktionen importieren
from data2day_exercises import *

## Daten einlesen

Polars unterstützt verschiedene Formate, um Daten in einen Dataframe einzulesen:

- CSV (`read_csv`, `read_csv_batched`)
- Apache Parquet (`read_parquet`)
- Databricks Delta (`read_delta`)
- SQL-Datenbanken (`read_database`, `read_database_uri`)
- JSON (`read_json`, `read_ndjson`)
- Microsoft Excel (`read_excel`)
- Apache OpenOffice (`read_ods`)
- Apache Avro (`read_avro`)
- Apache IPC (`read_ipc`, `read_ipc_stream`)
- Apache Iceberg

Wir lesen zuerst eine CSV-Datei ein.

In [ ]:
# Daten aus CSV-Datei einlesen
df = pl.read_csv(LOCAL_DATA_FILE_NAME)
df.head(2) # die ersten 2 Zeilen ausgeben

Die Datei enthält die täglichen Spotify-Charts. Folgende Informationen sind enthalten:

- `title`: Titel des Liedes
- `rank`: Platzierung in den Charts
- `date`: Tag an dem die Charts erhoben wurden
- `artist`: Band oder Künstler:innen, die das Lied performen
- `url`: URL unter der das Lied bei Spotify gehört werden kann
- `region`: Region oder Land für die die Charts erhoben werden
- `chart`: Bezeichnung bzw. Art der Charts
- `trend`: Entwicklung der Platzierung des Liedes gegenüber dem Vortag
- `streams`: Anzahl der Streams des Eintrags an dem Tag

In jeder Spalte steht unter dem Spaltennamen der Datentyp der Spalte. Die Datums-Spalte wurde als String (`str`) eingelesen, das lässt sich mit Hilfe der Option `try_parse_dates` korrigieren.

In [ ]:
# Daten aus CSV-Datei einlesen und Datums-Spalten parsen
df = pl.read_csv(LOCAL_DATA_FILE_NAME, try_parse_dates=True)
df.head(2) # die ersten 2 Zeilen ausgeben

## Projektion (Spalten auswählen)

Wenn nicht alle Spalten eines Dataframes benötigt werden, können mit der Methode `select` bestimmte Spalten ausgewählt werden.

In [ ]:
df.select("title", "artist", "url", "streams").head(2)

## Erweiterte Projektion (Spalten verändern oder hinzufügen)

Mit Hilfe von Ausdrücken (Expressions) können wir Spalten verändern oder auch neue Spalten hinzufügen.

Eine Spalte kann mit der Methode `pl.col` unter Angabe des Spalten-Namens referenziert werden.

In [ ]:
df.select("title", "artist", pl.col("url"), pl.col("streams")).head(2)

Damit wir kleinere Zahlen erhalten, können wir z.B. die Anzahl der Streams in Tausenden angeben.

In [ ]:
# Division mit dem "/" Operator konvertiert zu Floating-Point
df.select(pl.col("title"), pl.col("artist"), pl.col("url"), pl.col("streams")/1000).head(2)

In [ ]:
# Alternative mit "floordiv", gemischte Schreibweise mit reinen Spaltenname und pl.col ist auch möglich
df.select("title", "artist", "url", pl.col("streams").floordiv(1000)).head(2)

Neben Standard-Operatoren wie `+`, `-`, `*` und `/` steht eine Vielzahl von Ausdrücken für Berechnungen mit Zahlen steht zur Verfügung:

- https://pola-rs.github.io/polars/py-polars/html/reference/expressions/computation.html
- https://pola-rs.github.io/polars/py-polars/html/reference/expressions/operators.html

Für die Manipulation von Strings gibt es ebenfalls viele Funktionen. Diese werden über den eigenen Namensraum `str` angesprochen.

- https://pola-rs.github.io/polars/py-polars/html/reference/expressions/string.html

Ein Auswahl an häufig verwendeten Funktionen für Strings:

- `str.starts_with`, `str.ends_with`, `str.contains`
- `str.slice`
- `str.replace`
- `str.to_date`, `str.to_datetime`
- `str.split`
- `str.strip_chars`
- `str.n_chars`

In [ ]:
df.select(pl.col("title").str.to_uppercase(), "artist", "url", "streams").head(2)

Um nicht immer alle Spalten, die gar nicht transformiert werden, auflisten zu müssen, kann die Funktion `with_columns` genutzt werden.

In [ ]:
# with_columns entspricht select ergänzt um alle fehlenden Spalten
df.with_columns(pl.col("title").str.to_uppercase()).head(2)

Bis jetzt haben wir keine Spalten hinzugefügt. Eine neue Spalte entsteht, wenn wir einen Namen angeben, der bisher noch nicht existiert. Wir können dafür folgende Methoden verwenden:

- `alias` für einen vollständig neuen Namen
- `name.prefix`/`name.suffix` um den bestehenden Namen um ein Prefix/Suffix zu ergänzen

In [ ]:
# Track-ID aus der URL extrahieren
df.select("title", "url").with_columns(pl.col("url").str.slice(len("https://open.spotify.com/track/")).alias("trackId"), pl.col("title").str.to_uppercase().name.suffix("_uppercase")).head(2)

Wenn die gesamte Abfrage zu lang wird, sollte sie geklammert werden und mit Umbrüchen strukturiert werden. Auf diese Art und Weise entsteht eine typische "Abfrage-Pipeline", die von oben nach unten gelesen werden kann.

In [ ]:
(df
  .select("title", "url")
  .with_columns(
    pl.col("url").str.slice(len("https://open.spotify.com/track/")).alias("trackId"),
    pl.col("title").str.to_uppercase().name.suffix("_uppercase")
  )
  .head(2)
)

Wir können auch aggregierende Funktion wie `min`, `max`, `sum`, `mean`, `median`, etc. im `select` benutzen, wodurch wir eine Aggregation erhalten. Benutzen wir dabei eine Spalte mehr als einmal, müssen wir aufpassen einen passenden Namen zu vergeben. Entweder mit `alias` oder mit `suffix`.

In [ ]:
# Zeitraum bestimmen, für den Daten vorliegen
df.select(pl.col("date").min().name.suffix("_min"), pl.col("date").max().name.suffix("_max"))

## Selektion/Filtern

Mit Hilfe der Selektion lässt sich der Datensatz auf bestimmte Datensätze filtern.

Für einen schnellen Überblick lassen sich die Methoden `head`, `tail` und `sample` nutzen.

In [ ]:
# die ersten zwei Zeilen
df.head(2)

In [ ]:
# die letzten zwei Zeilen
df.tail(2)

In [ ]:
# zwei zufällig Zeilen (absolut mit Parameter "n" oder relativ mit Parameter "fraction")
df.sample(n=2)
df.sample(fraction=2/len(df)) # 2/362182 ≈ 0.000006 ist hier äquivalent zu n=2

Die Zeilen mit dem größten oder kleinsten Wert in einer Spalte lassen sich mit den Funktionen `top_k` und `bottom_k` auswählen.

In [ ]:
# der an einem Tag weltweit am meisten gestreamte Song auf Spotify: Easy On Me von Adele
df.top_k(1, by="streams")

Den können wir uns auch anhören.

In [ ]:
# spielt eine Vorschau des Songs mit Spotify. Bei mehreren Songs im Dataframe kann eine Zeilennummer angegebenen werden.
play_song(df.top_k(1, by="streams"))

Zeilen lassen sich mit der Methode `filter` und einem Boolschen-Ausdruck präzise auswählen. Wir können z.B. alle Datensätze einer bestimmten Künstlerin auswählen.

In [ ]:
# zwei Zeilen für die Sängerin "Adele"
# eq steht für equals
df.filter(pl.col("artist").eq("Adele")).head(2)

Eine Übersicht mit wichtigen Operatoren:
- Gleich (`==`): `eq`
- Ungleich (`!=`): `ne`
- Größer (`>`, `>=`): `gt`, `ge`
- Kleiner (`<`, `<=`): `lt`, `le`
- Zwischen: `is_between`
- Gleich einem aus einer Menge: `is_in`

Logische Ausdrücke lassen sich verknüpfen mit:
- Konjunktion/AND: `&`
- Disjunktion/OR: `|`
- Kontravalenz/XOR: `^`
- Negation/NOT: `~`

In [ ]:
# zwei Einträge für das Lied "Easy On Me" von Adele mit mehr als 3 Millionen Streams an einem Tag
df.filter(pl.col("artist").eq("Adele") & pl.col("title").eq("Easy On Me") & pl.col("streams").gt(3_000_000)).head(2)

Anstatt der Operatoren `eq` und `gt` wäre es auch möglich die Standard-Python-Operatoren `==` und `>` zu nutzen. Dann müssen aber alle logischen Teilausdrücke geklammert werden. Was ihr bevorzugt, ist am Ende Geschmackssache 😁

In [ ]:
df.filter(pl.col("artist").eq("Adele") & pl.col("title").eq("Easy On Me") & pl.col("streams").gt(3_000_000)).head(2)
df.filter((pl.col("artist") == "Adele") & (pl.col("title") == "Easy On Me") & (pl.col("streams") > 3_000_000)).head(2)

Für einen Vergleich mit einem konkreten Datum kann das Datum mit der Funktion `pl.date` erzeugt werden.

In [ ]:
# zwei Einträge für den 1. Mai 2017
df.filter(pl.col("date").eq(pl.date(2017, 5, 1))).head(2)

In [ ]:
# die Plätze 5 bis 10 für den 19. Juli 2018
df.filter(pl.col("date").eq(pl.date(2018, 7, 19)) & pl.col("rank").is_between(5, 10))

Wir können uns auch die täglichen Streams oder die Platzierung von Liedern mit einer Hilfsfunktion plotten lassen.

In [ ]:
some_song_df = df.filter(pl.col("artist").eq("Juice WRLD") & pl.col("title").eq("Lucid Dreams"))

In [ ]:
plot_streams(some_song_df)

In [ ]:
plot_rank(some_song_df)

## Übungen zu Projektion und Selektion

Die Übung kannst du direkt hier im Notebook machen. Für jede Übung gibt es ein Objekt (`q1`, `q2`, `q3`, ...), das die Frage, einen Hinweis, eine Antwortprüfung und die Lösung enthält.

In [ ]:
# Die Methode "question" gibt die Frage aus.
q0.question()

In [ ]:
# Dann gibt es immer eine Zelle mit einem Hinweis in welche Variablen die Lösung geschrieben werden sollte.
# Lege gerne weitere Zellen an, um deine Lösung genauer inspizieren zu können.
coole_firma = "BettercallPaul"

In [ ]:
# Die Methode "check" prüft eine Lösung.
q0.check(coole_firma)

In [ ]:
# Die Methode "hint" zeigt einen Hinweis an.
q0.hint()

In [ ]:
# Die Methode "solution" zeigt die Lösung.
q0.solution()

Jetzt bist du dran mit den richtigen Übungen!

### Frage 1

In [ ]:
q1.question()

In [ ]:
q1_df = ...

In [ ]:
q1.check(q1_df)
#q1.hint()
#q1.solution()

### Frage 2

In [ ]:
q2.question()

In [ ]:
q2_df = ...

In [ ]:
q2.check(q2_df)

### Frage 3

In [ ]:
q3.question()

In [ ]:
q3_df = ...

In [ ]:
q3.check(q3_df)

### Frage 4

In [ ]:
q4.question()

In [ ]:
rank_1 = ...
rank_200 = ...

In [ ]:
q4.check(rank_1, rank_200)

### Frage 5

In [ ]:
q5.question()

In [ ]:
q5_df = ...

In [ ]:
q5.check(q5_df)

### Frage 6

In [ ]:
q6.question()

In [ ]:
q6_df = ...

In [ ]:
q6.check(q6_df)

## Series

Normalerweise arbeiten wir immer auf einem Dataframe. Der Vollständigkeit halber sei aber erwähnt, dass es für die einzelnen Spalten den Datentyp `Series` gibt. Mit der Methode `get_column` oder dem Subset-Operator `[]` kann eine Spalte von einem Dataframe abgerufen werden.

In [ ]:
df.head(2).get_column("title")

In [ ]:
df.head(2)["title"]

In [ ]:
type(df.get_column("title"))

## Datentypen

Polars kann viele verschiedene Daten in einer Spalte schreiben.

### Zahlen und boolesche Werte

- `Int8`, `Int16`, `Int32`, `Int64`: Ganzzahl
- `Float32`, `Float64`: Fließkommazahl
- `UInt8`, `UInt16`, `UInt32`, `UInt64`: natürliche Zahl (ohne Vorzeichen)
- `Decimal`: 128-Bit Fließkommazahl mit hoher Präzision, experimentell
- `Boolean`: logischer Wahrheitswert

Zahlen werden in Polars als 64-Bit Datentypen angelegt, wenn nicht anders angegeben.

Ein Spalte lässt sich mit der Funktion `cast` in einen anderen Datentyp konvertieren, um z.B. Speicherplatz zu sparen.

In [ ]:
# Standard-Datentyp ist Int64 bzw. Float64 für Zahlen
df.select(pl.col("streams")).head(2)

In [ ]:
# wirft einen Fehler, weil einige Werte für Int16 zu groß sind
try:
    df.select(pl.col("streams").cast(pl.Int16)).head(2)
except pl.ComputeError as e:
    print(e.args)


In [ ]:
# wirft keinen Fehler, weil Int32 ausreichend groß ist
df.select(pl.col("streams").cast(pl.Int32)).head(2)

Aber Achtung bevor du jetzt alles zum kleinstmöglichen Datentyp konvertierst: bei 32-Bit Datentypen kann es bei Berechnungen zu Überlaufen kommen, für die keine Warnung ausgegeben wird!

In [ ]:
print(f'Anzahl aller Streams mit Int64 ist {df.select(pl.col("streams").sum()).item()}')
print(f'Anzahl aller Streams mit Int32 ist {df.select(pl.col("streams").cast(pl.Int32).sum()).item()}')

Mit der Methode `shrink_dtype` lässt sich der Speicherverbrauch ein Stück weit auch automatisiert verringern. Dabei wird aber nie von "signed" zu "unsigned" Datentypen gewechselt, auch wenn keine negativen Daten vorhanden sind.

In [ ]:
df.select(pl.col("rank").shrink_dtype()).head(2)

### Datum und Zeit

- `Date`: Datum
- `Time`: Uhrzeit
- `Datetime`: Zeitpunkt
- `Duration`: Zeitdauer

Komponenten aus Daten und Zeiten lassen sich über den Kontext `dt` extrahieren.

In [ ]:
(df
    .select("date")
    .with_columns(
        pl.col("date").dt.year().alias("year"),
        pl.col("date").dt.quarter().alias("quarter"),
        pl.col("date").dt.month().alias("month"),
        pl.col("date").dt.week().alias("week"),
        pl.col("date").dt.weekday().alias("weekday"), # Monday == 1, Sunday == 7
        pl.col("date").dt.day().alias("day"),
    )
    .sample(5)
)

Wir können auch Daten voneinander abziehen oder eine Zeitspanne addieren oder abziehen (`offset_by`).

In [ ]:
(df
    .select("date")
    .with_columns(
        (pl.col("date").dt.month_end() - pl.col("date")).alias("days_till_month_end"),
        pl.col("date").dt.offset_by("1w").alias("same_day_next_week")
    )
    .sample(5)
)

In [ ]:
(df
    .filter(pl.col("date").eq(pl.col("date").dt.month_end()))
    .select("date", pl.col("artist"))
    .sample(5)
)

### Zeichenketten

- `Utf8`: beliebige Zeichenkette
- `Categorical`: Zeichenkette kodiert als Kategorie

### Strukturen

- `List`: Liste mit variabler Länge je Zeile
- `Array`: Liste mit fester Länger in allen Zeilen
- `Struct`: benamte Felder

### Sonstiges

- `Binary`: binäre Daten
- `Object`: beliebiges Python-Objekt

## Sortieren

Mit der Methode `sort` lassen sich Dataframes einfach sortieren.

In [ ]:
df.sort("rank").head(3)

In [ ]:
df.sort("streams", descending=True).head(3)

In [ ]:
df.sort(["rank", "streams"], descending=[False, True]).head(3)

## Daten schreiben

Ein Dataframe kann mit den Methoden `write_*` in verschiedenen Formaten in eine Datei geschrieben werden.

In [ ]:
df_2020 = df.filter(pl.col("date").dt.year().eq(2020))

In [ ]:
# als CSV (ca. 9 MB)
df_2020.write_csv("2020_write_test.csv", )

In [ ]:
# als komprimierte CSV (ca. 2 MB)
import gzip

with gzip.open("2020_write_test.csv.gz", "wb") as f:
    df_2020.write_csv(f)

In [ ]:
# als Apache Parquet (ca. 1 MB)
df_2020.write_parquet("2020_write_test.parquet")

In [ ]:
!ls -l 2020_write_test*

## Optionale Übungen

### Frage 7

In [ ]:
q7.question()

In [ ]:
q7_df = ...

In [ ]:
q7.check(q7_df)

### Frage 8

In [ ]:
q8.question()

In [ ]:
q8_monday = ...
q8_friday = ...

In [ ]:
q8.check(q8_monday, q8_friday)

### Frage 9

In [ ]:
q9.question()

In [ ]:
q9_df = ...

In [ ]:
q9.check(q9_df)

### Frage 10

In [ ]:
q10.question()

In [ ]:
q10_df = ...

In [ ]:
q10.check(q10_df)

### Frage 11

In [ ]:
q11.question()

In [ ]:
q11_ohne_zedd = ...
q11_mit_zedd = ...

In [ ]:
q11.check(q11_ohne_zedd, q11_mit_zedd)

### Frage 12

In [ ]:
q12.question()

In [ ]:
q12_df = ...

In [ ]:
q12.check(df, q12_df)